In [1]:
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession

In [14]:
spark = SparkSession.builder.master('local[1]').appName('Teste.com').getOrCreate()

In [15]:
spark

In [20]:
df = spark.read.csv("/Users/aurelianosancho/Google Drive/Spark/data_o.csv",header=True)

In [21]:
df.printSchema()

root
 |-- valence: string (nullable = true)
 |-- year: string (nullable = true)
 |-- acousticness: string (nullable = true)
 |-- artists: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- duration_ms: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- explicit: string (nullable = true)
 |-- id: string (nullable = true)
 |-- instrumentalness: string (nullable = true)
 |-- key: string (nullable = true)
 |-- liveness: string (nullable = true)
 |-- loudness: string (nullable = true)
 |-- mode: string (nullable = true)
 |-- name: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- speechiness: string (nullable = true)
 |-- tempo: string (nullable = true)



In [23]:
df.show(2)

+-------+----+------------+--------------------+------------------+-----------+------+--------+--------------------+----------------+---+--------+--------+----+--------------------+----------+------------+-----------+-----------------+
|valence|year|acousticness|             artists|      danceability|duration_ms|energy|explicit|                  id|instrumentalness|key|liveness|loudness|mode|                name|popularity|release_date|speechiness|            tempo|
+-------+----+------------+--------------------+------------------+-----------+------+--------+--------------------+----------------+---+--------+--------+----+--------------------+----------+------------+-----------+-----------------+
| 0.0594|1921|       0.982|['Sergei Rachmani...|             0.279|     831667| 0.211|       0|4BJqT0PrAfrxzMOxy...|           0.878| 10|   0.665| -20.096|   1|Piano Concerto No...|         4|        1921|     0.0366|           80.954|
|  0.963|1921|       0.732|      ['Dennis Day']|0.819000

In [11]:
dd = pd.read_csv("/Users/aurelianosancho/Google Drive/Spark/data_o.csv")
dd.head(3)

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.961,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339


In [33]:
# Rename Column on DataFrame
df = df.withColumnRenamed("year","Ano")
df.printSchema()

root
 |-- valence: string (nullable = true)
 |-- Ano: string (nullable = true)
 |-- acousticness: string (nullable = true)
 |-- artists: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- duration_ms: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- explicit: string (nullable = true)
 |-- id: string (nullable = true)
 |-- instrumentalness: string (nullable = true)
 |-- key: string (nullable = true)
 |-- liveness: string (nullable = true)
 |-- loudness: string (nullable = true)
 |-- mode: string (nullable = true)
 |-- name: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- speechiness: string (nullable = true)
 |-- tempo: string (nullable = true)



In [34]:
from pyspark.sql.functions import col

In [36]:
df = df.withColumn("Nova",col("popularity")* -1)
df.printSchema()

root
 |-- valence: string (nullable = true)
 |-- year: string (nullable = true)
 |-- acousticness: string (nullable = true)
 |-- artists: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- duration_ms: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- explicit: string (nullable = true)
 |-- id: string (nullable = true)
 |-- instrumentalness: string (nullable = true)
 |-- key: string (nullable = true)
 |-- liveness: string (nullable = true)
 |-- loudness: string (nullable = true)
 |-- mode: string (nullable = true)
 |-- name: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- speechiness: string (nullable = true)
 |-- tempo: string (nullable = true)
 |-- Nova: double (nullable = true)



In [38]:
df.filter(df.artists.startswith("a")).show()

+-------+----+------------+-------+------------+-----------+------+--------+---+----------------+---+--------+--------+----+----+----------+------------+-----------+-----+----+
|valence|year|acousticness|artists|danceability|duration_ms|energy|explicit| id|instrumentalness|key|liveness|loudness|mode|name|popularity|release_date|speechiness|tempo|Nova|
+-------+----+------------+-------+------------+-----------+------+--------+---+----------------+---+--------+--------+----+----+----------+------------+-----------+-----+----+
+-------+----+------------+-------+------------+-----------+------+--------+---+----------------+---+--------+--------+----+----+----------+------------+-----------+-----+----+



In [40]:
df.filter(df.artists.contains("N")).show(2)

+-------+----+------------+--------------------+------------------+-----------+-------------------+--------+--------------------+----------------+---+--------+-------------------+----+------------+----------+------------+-----------+-------+----+
|valence|year|acousticness|             artists|      danceability|duration_ms|             energy|explicit|                  id|instrumentalness|key|liveness|           loudness|mode|        name|popularity|release_date|speechiness|  tempo|Nova|
+-------+----+------------+--------------------+------------------+-----------+-------------------+--------+--------------------+----------------+---+--------+-------------------+----+------------+----------+------------+-----------+-------+----+
| 0.0394|1921|       0.961|['KHP Kridhamarda...|             0.328|     500062|              0.166|       0|1o6I8BglA6ylDMrIE...|           0.913|  3|   0.101|             -14.85|   1|   Gati Bali|         5|        1921|     0.0339|110.339|-5.0|
|  0.196|192

In [27]:
df.createOrReplaceTempView("Tabela1")

In [30]:
df2 = spark.sql("SELECT artists,popularity from Tabela1")

In [31]:
df2.show()

+--------------------+----------+
|             artists|popularity|
+--------------------+----------+
|['Sergei Rachmani...|         4|
|      ['Dennis Day']|         5|
|['KHP Kridhamarda...|         5|
|    ['Frank Parker']|         3|
|      ['Phil Regan']|         2|
|['KHP Kridhamarda...|         6|
|  ['John McCormack']|         4|
|['Sergei Rachmani...|         2|
| ['Ignacio Corsini']|         0|
|         ['Fortugé']|         0|
|['Maurice Chevali...|         0|
| ['Ignacio Corsini']|         0|
|         ['Georgel']|         0|
| ['Mehmet Kemiksiz']|         0|
|      ['Zay Gatsby']|         0|
|['Sergei Rachmani...|         4|
|      ['Phil Regan']|         1|
|['Hector Berlioz'...|         0|
|         ['THE GUY']|         0|
|['Christopher Lyn...|         0|
+--------------------+----------+
only showing top 20 rows

